# Control de versiones con git

**Curso**: Ciencia de Datos en Python

**Catedrático**: Ing. Luis Leal

**Estudiante**: Dany Rafael Díaz Lux (21000864)

**Enunciado**: Tomar la [lección del MIT de git](https://missing.csail.mit.edu/2020/version-control/) y en un jupyter notebook entregar un ensayo acerca de git, es decir: un resumen de lo que ha aprendido, conclusiones, comentarios y ejemplos propios, esto usando markdown en el notebook , para ejemplificar cualquier comando de git pueden usar [magic cells](https://jakevdp.github.io/PythonDataScienceHandbook/01.03-magic-commands.html), un tipo especial de celdas de jupyter que nos permite escribir comandos que no son de python si no de nuestro sistema.

**Fecha de entrega**: miércoles 10 de febrero.

## Resumen

## Comentarios personales

## Conclusiones

## Ejemplos propios